In [1]:
import random
import pickle
import os
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import Sequence
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
import tensorflow as tf
import cv2

Using TensorFlow backend.


In [2]:
# Directories declaration and initiating the patient names

sampled_pickles = '/home/omar/Desktop/sampled_output/'


patients = os.listdir(sampled_pickles)
random.shuffle(patients)
# training_patients = patients[:26]
training_patients = ['TCGA-76-6282.pickle', 'TCGA-76-4932.pickle', 'TCGA-02-0046.pickle', 'TCGA-06-0138.pickle', 'TCGA-76-6657.pickle', 'TCGA-76-4935.pickle', 'TCGA-19-5960.pickle', 'TCGA-76-6280.pickle', 'TCGA-02-0006.pickle', 'TCGA-19-1789.pickle', 'TCGA-02-0009.pickle', 'TCGA-19-0963.pickle', 'TCGA-02-0086.pickle', 'TCGA-06-0149.pickle', 'TCGA-06-0179.pickle', 'TCGA-06-2570.pickle', 'TCGA-02-0075.pickle', 'TCGA-12-0616.pickle', 'TCGA-02-0085.pickle', 'TCGA-06-5417.pickle', 'TCGA-06-0145.pickle', 'TCGA-08-0355.pickle', 'TCGA-02-0054.pickle', 'TCGA-19-2624.pickle', 'TCGA-06-0177.pickle', 'TCGA-02-0106.pickle']
# test_patients = patients[26:29]
test_patients = ['TCGA-08-0509.pickle', 'TCGA-08-0520.pickle', 'TCGA-06-6389.pickle']
image_width = 256
image_height = 256
# batch_size= 16
current_dir = '/home/omar/Desktop/Generalization/Model/'

In [3]:
# Preparing data npy files and loading model-suitable images
def view_original(set_of_patients, npy_train, npy_mask):
    for patient in set_of_patients:
        path = sampled_pickles + patient
        pickle_file = open("%s"%(path), "rb")
        patient_data = pickle.load(pickle_file)
        npy_array=0
        
        for scan in range(len(patient_data)):
            if scan!=4:
                for array in range(5):
                    plt.figure(figsize=(15,15))
                    plt.subplot(221)
                    plt.imshow(np.squeeze(patient_data[scan][array]), 'gray')
                    plt.subplot(222)
                    plt.imshow(np.squeeze(patient_data[4][scan][array]), 'gray')
                    plt.subplot(223)
                    plt.imshow(npy_train[npy_array], 'gray')
                    plt.subplot(224)
                    plt.imshow(npy_mask[npy_array], 'gray')
                    npy_array+=1
        

def normalize(input_image, input_mask):
  input_image = tf.cast(input_image, tf.float32) / 255.0
  input_mask -= 1
  return input_image, input_mask


def npy_generator(training, testing):
    path = sampled_pickles
    out_dir = '/home/omar/Desktop/Generalization/Model/TF_SEG_TEST/'
    os.chdir(out_dir)
    img_height = 256
    img_width = 256
    total_scans = 0
    total_scans_test = 0
    
    for patient in training:
        patient_path = path + patient
        pickle_file = open("%s"%(patient_path), "rb")
        patient_data = pickle.load(pickle_file)
        total_scans += len(patient_data[0]) + len(patient_data[1]) + len(patient_data[2]) + len(patient_data[3])
    
    
    for patient in testing:
        patient_path = path + patient
        pickle_file = open("%s"%(patient_path), "rb")
        patient_data = pickle.load(pickle_file)
        total_scans_test += len(patient_data[0]) + len(patient_data[1]) + len(patient_data[2]) + len(patient_data[3])
        
    
    imgs_train = np.ndarray((total_scans, img_width, img_height), dtype = np.float32)
    masks_train = np.ndarray((total_scans, img_width, img_height), dtype = np.float32)
    
    imgs_test = np.ndarray((total_scans_test, img_width, img_height), dtype = np.float32)
    ground_truth = np.ndarray((total_scans, img_width, img_height), dtype = np.float32)
    
    indexer = 0
    for patient in training:
        print('Current Patient: %s' % (patient))
        patient_path = path + patient
        pickle_file = open("%s"%(patient_path), "rb")
        patient_data = pickle.load(pickle_file)
        for scan in range(len(patient_data)):
            if scan != 4:
                for array in range(len(patient_data[scan])):
                    img = patient_data[scan][array]
                    img_mask = patient_data[4][scan][array]
                    imgs_train[indexer] = img
                    masks_train[indexer] = img_mask
                    indexer+=1
    np.save('imgs_train.npy', imgs_train)
    np.save('img_masks_train.npy', masks_train)
    
    indexer_test = 0
    for patient in testing:
        print('Current Patient: %s' % (patient))
        patient_path = path + patient
        pickle_file = open("%s"%(patient_path), "rb")
        patient_data = pickle.load(pickle_file)
        for scan in range(len(patient_data)):
            if scan != 4:
                for array in range(len(patient_data[scan])):
                    img = patient_data[scan][array]
                    img_mask = patient_data[4][scan][array]
                    imgs_test[indexer_test] = img
                    ground_truth[indexer_test] = img_mask
                    indexer_test+=1
                    
    np.save('imgs_test.npy', imgs_test)
    np.save('ground_truth.npy', ground_truth)
    
    print('Training and testing npy files generated.')

    

def load_image_train():
    path = '/home/omar/Desktop/Generalization/Model/TF_SEG_TEST/'
    training_dataset = path + 'imgs_train.npy'
    mask_dataset = path + 'img_masks_train.npy'
    imgs = np.load(training_dataset)
    masks = np.load(mask_dataset)
#     for arr in range(len(imgs)):
#         print(np.shape(imgs[arr]))
# #         imgs[arr] = cv2.resize(imgs[arr], (128,128))
#         masks[arr] = cv2.resize(masks[arr], (128,128))
    print('Loaded training dataset')
    return imgs, masks


def load_image_test():
    path = '/home/omar/Desktop/Generalization/Model/TF_SEG_TEST/'
    test_dataset = path + 'imgs_test.npy'
    ground_truth = path + 'ground_truth.npy'
    imgs = np.load(test_dataset)
    masks = np.load(ground_truth)
#     for arr in range(len(imgs)):
#         imgs[arr] = np.resize(imgs[arr], (128,128))
#         masks[arr] = np.resize(masks[arr], (128,128))
    print('Loaded testing dataset')
    return imgs, masks

def batch_norm(npy_train_imgs, npy_train_masks, npy_test_imgs, npy_test_masks):
    final_training_images = np.ndarray((npy_train_imgs.shape[0], 256, 256), dtype=np.float32)
    final_training_masks = np.ndarray((npy_train_masks.shape[0], 256, 256), dtype=np.float32)
    final_testing_images = np.ndarray((npy_test_imgs.shape[0], 256, 256), dtype=np.float32)
    final_testing_masks = np.ndarray((npy_test_masks.shape[0], 256, 256), dtype=np.float32)
    
    
    for image in range(len(npy_train_imgs)):
        final_training_images[image] = tf.cast(npy_train_imgs[image], tf.float32) / 255.0
        final_training_masks[image] = npy_train_masks[image]-1
    
    for image in range(len(npy_test_imgs)):
        final_testing_images[image] = tf.cast(npy_test_imgs[image], tf.float32) / 255.0
        final_testing_masks[image] = npy_test_masks[image]-1
#     for image in range(len(input_images)):
#         train_image = tf.cast(input_images[image], tf.float32) / 255.0
#         train_mask = input_masks[image] - 1
#         input_images[image] = train_image
#         input_masks[image] = train_mask
    return final_training_images, final_training_masks, final_testing_images, final_testing_masks

In [8]:
batch_size = 64
steps_per_epoch = 26 // batch_size
output_channels = 2

In [9]:
# Running Helper Functions for data generation
print('-'*30)
print('Generating npy files for training and testing datasets')
npy_generator(training_patients, test_patients)
print('-'*30)
print('Loading training and testing datasets')
train_imgs, train_masks = load_image_train()
test_imgs, ground_truth = load_image_test()
print('-'*30)

------------------------------
Generating npy files for training and testing datasets
Current Patient: TCGA-76-6282.pickle
Current Patient: TCGA-76-4932.pickle
Current Patient: TCGA-02-0046.pickle
Current Patient: TCGA-06-0138.pickle
Current Patient: TCGA-76-6657.pickle
Current Patient: TCGA-76-4935.pickle
Current Patient: TCGA-19-5960.pickle
Current Patient: TCGA-76-6280.pickle
Current Patient: TCGA-02-0006.pickle
Current Patient: TCGA-19-1789.pickle
Current Patient: TCGA-02-0009.pickle
Current Patient: TCGA-19-0963.pickle
Current Patient: TCGA-02-0086.pickle
Current Patient: TCGA-06-0149.pickle
Current Patient: TCGA-06-0179.pickle
Current Patient: TCGA-06-2570.pickle
Current Patient: TCGA-02-0075.pickle
Current Patient: TCGA-12-0616.pickle
Current Patient: TCGA-02-0085.pickle
Current Patient: TCGA-06-5417.pickle
Current Patient: TCGA-06-0145.pickle
Current Patient: TCGA-08-0355.pickle
Current Patient: TCGA-02-0054.pickle
Current Patient: TCGA-19-2624.pickle
Current Patient: TCGA-06-0

In [10]:
# view_original(training_patients,train_imgs, train_masks)

In [11]:
# Normalizing the dataset
final_train_norm = tf.cast(train_imgs, tf.float32) / 255.0
final_test_norm = tf.cast(test_imgs, tf.float32) / 255.0
final_test_imgs = test_imgs-1
final_ground_truth = ground_truth - 1

In [ ]:
# WORK IN PROGRESS... MATCH THIS MODEL WITH THE PREPARED DATSET
# def UNet(pretrained_weights=None, input_size = (256,256,1)):
#     inputs = Input(input_size)
#     conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
#     conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
#     pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
#     conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
#     conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
#     pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
#     conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
#     conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
#     pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
#     conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
#     conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
#     drop4 = Dropout(0.5)(conv4)
#     pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

#     conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
#     conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
#     drop5 = Dropout(0.5)(conv5)

#     up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
#     merge6 = concatenate([drop4,up6], axis = 3)
#     conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
#     conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

#     up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
#     merge7 = concatenate([conv3,up7], axis = 3)
#     conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
#     conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

#     up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
#     merge8 = concatenate([conv2,up8], axis = 3)
#     conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
#     conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

#     up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
#     merge9 = concatenate([conv1,up9], axis = 3)
#     conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
#     conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
#     conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
#     conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

#     model = Model(input = inputs, output = conv10)

#     model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])